In [ ]:
import scipy.io

filepath='data/raw/'
matfile = scipy.io.loadmat(filepath)
print(matfile.keys())

In [2]:
import os
def find_all_mat_files_in_order(root_dir):   
    mat_files = []
    for dirpath, _, filenames in sorted(os.walk(root_dir)):
        for file in filenames:
            if file.endswith('.mat'):
                mat_files.append(os.path.join(dirpath, file))
    return mat_files

In [11]:
import math

def load_torque_to_hp(torque_nm, rpm):
    power_watts = (2 * math.pi * torque_nm * rpm) / 60
    power_hp = power_watts / 745.7
    return power_hp


In [27]:
import os
import re

dir = 'data/raw/paderborn/K001'

list_files = find_all_mat_files_in_order(dir)
for file in list_files:
    key = re.findall(r'_K[0AI]\d{2}_', file)[0][2]
    dataset_name = 'Paderborn'
    filename = os.path.basename(file).split('.')[0]
    label = {'0':'N', 'I':'I', 'A':'O'}[key]
    sampling_rate = 64000
    rpm = {'09':'900', '15':'1500'}[re.findall(r'N(\d{2})', file)[0]]
    load = {'04':'400', '10':'1000'}[re.findall(r'_F(\d{2})_', file)[0]]
    load_torque = re.findall(r'_M(\d{2})_', file)[0][1]
    hp = round(load_torque_to_hp(int(load_torque)*0.1, int(rpm)), 2)
    extent_damage = '000'
    damage_method = 'no'
    bearing_type = 6203
    condition_bearing_health = {'0':'healthy', 'A':'faulty', 'I':'faulty'}[key]
    print(f'{dataset_name},{filename},{label},{sampling_rate},{rpm},{hp},{load},{extent_damage},{damage_method},{bearing_type},{condition_bearing_health}')

Paderborn,N15_M07_F04_K001_1,N,64000,1500,0.15,400,000,no,6203,healthy
Paderborn,N09_M07_F10_K001_6,N,64000,900,0.09,1000,000,no,6203,healthy
Paderborn,N15_M07_F10_K001_3,N,64000,1500,0.15,1000,000,no,6203,healthy
Paderborn,N15_M07_F04_K001_4,N,64000,1500,0.15,400,000,no,6203,healthy
Paderborn,N15_M01_F10_K001_2,N,64000,1500,0.02,1000,000,no,6203,healthy
Paderborn,N15_M07_F04_K001_16,N,64000,1500,0.15,400,000,no,6203,healthy
Paderborn,N15_M07_F04_K001_19,N,64000,1500,0.15,400,000,no,6203,healthy
Paderborn,N15_M01_F10_K001_12,N,64000,1500,0.02,1000,000,no,6203,healthy
Paderborn,N15_M07_F10_K001_17,N,64000,1500,0.15,1000,000,no,6203,healthy
Paderborn,N09_M07_F10_K001_15,N,64000,900,0.09,1000,000,no,6203,healthy
Paderborn,N15_M07_F10_K001_2,N,64000,1500,0.15,1000,000,no,6203,healthy
Paderborn,N15_M07_F04_K001_10,N,64000,1500,0.15,400,000,no,6203,healthy
Paderborn,N15_M07_F10_K001_12,N,64000,1500,0.15,1000,000,no,6203,healthy
Paderborn,N15_M01_F10_K001_19,N,64000,1500,0.02,1000,000,no,6203

In [24]:
import re
re.findall(r'angelo(\d{3})', 'adriano123_rocha_angelo123.com')

['123']